# Tutorial5: 大模型微调

本节旨在通过 [cpm-bee-2b](https://huggingface.co/openbmb/cpm-bee-2b) 模型来展示大模型微调。

分以下几步来实现：
1. 环境安装
2. 下载模型
3. 上传数据集
4. 模型微调

## 1. 环境安装

我们默认用户已经按照 [tutorial_scow_for_ai](../tutorial_scow_for_ai.md) 在 “交互式应用” 中创建了  Jupyter Lab 应用，并已经安装了 conda。现在需要在命令行中创建环境并注册 ipykernal：

```bash
conda create --name cpm-bee python=3.9
conda activate cpm-bee
# 安装内核
conda install ipykernel
# 注册内核
python -m ipykernel install --user --name=cpm-bee --display-name="cpm-bee"
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
pip install deepspeed accelerate==0.30.1 transformers==4.33.2
```

（ pytorch 版本需与 cuda 版本对应，请查看版本对应网站：https://pytorch.org/get-started/previous-versions ，通过 nvidia-smi 命令可查看 cuda 版本）

打开本 .ipynb 文件并选择 kernel 为 cpm-bee。硬件资源建议使用1张GPU运行。

CUDA Version: 12.1; Torch Version: 2.3.1

## 2. 下载模型

CPM-Bee 是一个完全开源、可商用的中英双语基础模型，拥有一百亿参数的容量。我们这里选用的是较小参数的 cpm-bee-2b 模型。在联网的命令行中执行以下命令，命令执行位置在当前文件所在的文件夹。

```bash
# 如果以下目录存在， 可以直接复制:
cp -r /lustre/public/tutorial/models/cpm-bee-2b/ ./

# 否则请自行下载：
export HF_ENDPOINT=https://hf-mirror.com
huggingface-cli download --resume-download openbmb/cpm-bee-2b --local-dir cpm-bee-2b
```

## 3. 上传数据集

可以自行上传数据，我们这里准备了数据 bee_data

## 4. 微调

```bash
accelerate config
```

生成的配置文件在 ~/.cache/huggingface/accelerate/default_config.yaml 下，内容为：

```yaml
compute_environment: LOCAL_MACHINE                                 
debug: false
distributed_type: MULTI_GPU
downcast_bf16: 'no'
enable_cpu_affinity: false
gpu_ids: all
machine_rank: 0
main_training_function: main
mixed_precision: fp16
num_machines: 1
num_processes: 4
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false
```

In [6]:
code = '''import json
from torch.utils.data import Dataset

class CPMDataset(Dataset):
    def __init__(self, jsonl_file):
        self.data = []
        with open(jsonl_file, 'r', encoding='utf-8') as file:
            for line in file:
                # 解析每一行 JSON 数据
                item = json.loads(line)
                # 提取需要的字段
                input_text = item['input']
                answer = item['<ans>']
                # 将数据添加到列表中
                self.data.append({"input": input_text, "<ans>": answer})

    def __len__(self):
        # 返回数据集的大小
        return len(self.data)

    def __getitem__(self, idx):
        # 返回格式化的数据
        return self.data[idx]
'''

with open('data_prepare.py', 'w', encoding='utf-8') as f:
    f.write(code)

In [1]:
%%bash

echo '''from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import Accelerator
from data_prepare import CPMDataset
from torch.utils.data import DataLoader
import torch
import time

# 数据准备
trainset = CPMDataset("bee_data/train.jsonl")  
train_loader = DataLoader(trainset, batch_size=1)

# 模型
model_path = "cpm-bee-1b"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

# 训练
optimizer = torch.optim.Adam(model.parameters())

accelerator = Accelerator()
model, optimizer, train_loader = accelerator.prepare(
    model, optimizer, train_loader
)

# 计时
total_time = 0

for iter, data in enumerate(train_loader):
    
    model.train()
    
    step_start = time.perf_counter()
    
    # 训练
    optimizer.zero_grad()
    input_encoded = tokenizer.prepare_for_finetune(data, max_length=512)
    outputs = model(**input_encoded)
    loss = outputs.loss
    accelerator.backward(loss)
    optimizer.step()
    
    step_time = time.perf_counter() - step_start
    total_time += step_time
    
    # 仅在主进程输出
    if accelerator.is_main_process:
        print(f"Step {iter}, Loss: {loss.item():.4f}, Time per step: {step_time:.4f} s")

    
# 仅在主进程输出
if accelerator.is_main_process:
    print("Training done")
    print(f"Total training time: {total_time:.2f} s")
    print(f"Average time per step: {total_time / (iter + 1):.4f} s")
''' > cpm-bee-train.py

在命令行中执行：

```bash
accelerate launch cpm-bee-train.py
```

---

> 作者: 黎颖; 龙汀汀
>
> 联系方式: yingliclaire@pku.edu.cn;   l.tingting@pku.edu.cn